# tf.data

: tensorflow를 이용하여 데이터를 전처리하기 위해 파이프라인 구성 -> 셔플, 배치, 반복 등 다양한 기능

** activate tensorflow (conda env)

In [1]:
import os
import numpy as np
import tensorflow as tf #텐서플로우 모듈 불러오기

from tensorflow.keras import preprocessing

In [2]:
# step1: 리스트로 만들어 text 수치화

samples = ['너 오늘 이뻐 보인다', 
           '나는 오늘 기분이 더러워', 
           '끝내주는데, 좋은 일이 있나봐', 
           '나 좋은 일이 생겼어', 
           '아 오늘 진짜 짜증나', 
           '환상적인데, 정말 좋은거 같아']

label = [[1], [0], [1], [1], [0], [1]]

In [3]:
MAX_LEN = 4

In [4]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)    # 토큰을 딕셔너리 형태로 make
sequences = tokenizer.texts_to_sequences(samples)    # 글자를 숫자로 대체

sequences = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_LEN, padding='post')

word_index = tokenizer.word_index    # 딕셔너리 확인

In [5]:
print("수치화된 텍스트 데이터: \n",sequences)
print("각 단어의 인덱스: \n", word_index) 
print("라벨: ", label) 

수치화된 텍스트 데이터: 
 [[ 4  1  5  6]
 [ 7  1  8  9]
 [10  2  3 11]
 [12  2  3 13]
 [14  1 15 16]
 [17 18 19 20]]
각 단어의 인덱스: 
 {'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}
라벨:  [[1], [0], [1], [1], [0], [1]]


In [6]:
# tf.data 사용하기: 입력 함수 구현 가능
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))    # sequences와 label을 묶어서 사용할 수 있게 해줌
iterator = dataset.make_one_shot_iterator()   # 데이터를 하나씩 사용할 수 있게 만들어 줌
next_data = iterator.get_next()   # 데이터가 하나씩 나오게 되는 구조

In [7]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))


In [8]:
BATCH_SIZE = 2   # batch size: 한 번에 사용할 데이터 단위

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.batch(BATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

In [9]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([[4, 1, 5, 6],
       [7, 1, 8, 9]]), array([[1],
       [0]]))
(array([[10,  2,  3, 11],
       [12,  2,  3, 13]]), array([[1],
       [1]]))
(array([[14,  1, 15, 16],
       [17, 18, 19, 20]]), array([[0],
       [1]]))


In [10]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.shuffle(len(sequences))   # data를 임의의 순서대로 섞어주는 기능 # 고정된 값은 아님
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

In [11]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([12,  2,  3, 13]), array([1]))
(array([17, 18, 19, 20]), array([1]))
(array([10,  2,  3, 11]), array([1]))
(array([14,  1, 15, 16]), array([0]))


In [12]:
EPOCH = 2

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.repeat(EPOCH)  # epoch의 수만큼 dataset 반복 생성
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

In [13]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))
(array([4, 1, 5, 6]), array([1]))
(array([7, 1, 8, 9]), array([0]))
(array([10,  2,  3, 11]), array([1]))
(array([12,  2,  3, 13]), array([1]))
(array([14,  1, 15, 16]), array([0]))
(array([17, 18, 19, 20]), array([1]))


In [16]:
def mapping_fn(X, Y=None):  # label을 제외한 입력값을 딕셔너리 형태로 만듬
    input = {'x': X}
    label = Y
    return input, label

In [17]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)   # 모든 dataset에 한 번에 적용
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

In [18]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

({'x': array([4, 1, 5, 6])}, array([1]))
({'x': array([7, 1, 8, 9])}, array([0]))
({'x': array([10,  2,  3, 11])}, array([1]))
({'x': array([12,  2,  3, 13])}, array([1]))
({'x': array([14,  1, 15, 16])}, array([0]))
({'x': array([17, 18, 19, 20])}, array([1]))


In [19]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
dataset = dataset.shuffle(len(sequences))
dataset = dataset.batch(BATCH_SIZE) 
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

In [20]:
with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

({'x': array([[12,  2,  3, 13],
       [ 4,  1,  5,  6]])}, array([[1],
       [1]]))
({'x': array([[17, 18, 19, 20],
       [10,  2,  3, 11]])}, array([[1],
       [1]]))
({'x': array([[14,  1, 15, 16],
       [ 7,  1,  8,  9]])}, array([[0],
       [0]]))


In [21]:
BATCH_SIZE = 2
EPOCH = 2

def mapping_fn(X, Y=None):
    input = {'x': X}
    label = Y
    return input, label

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
dataset = dataset.shuffle(len(sequences))
dataset = dataset.batch(BATCH_SIZE) 
dataset = dataset.repeat(EPOCH)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

({'x': array([[10,  2,  3, 11],
       [12,  2,  3, 13]])}, array([[1],
       [1]]))
({'x': array([[ 4,  1,  5,  6],
       [14,  1, 15, 16]])}, array([[1],
       [0]]))
({'x': array([[17, 18, 19, 20],
       [ 7,  1,  8,  9]])}, array([[1],
       [0]]))
({'x': array([[17, 18, 19, 20],
       [12,  2,  3, 13]])}, array([[1],
       [1]]))
({'x': array([[14,  1, 15, 16],
       [ 7,  1,  8,  9]])}, array([[0],
       [0]]))
({'x': array([[ 4,  1,  5,  6],
       [10,  2,  3, 11]])}, array([[1],
       [1]]))
